# Scene Segmentation Model

In [7]:
import inspect
from viper_toolkit import Dissect
from model_server import ViperModel, NeuralNetworkLoader
from scene_segmentation_module import SceneSegmentationModel

## Scene Segmentation Class

The Scene Segmentation Model is a class of NeuralNetworkLoader models which performs scene segmentation on the image classifying the pixels in four classes:

* Roadway
* curb
* Backgroun
* Marker


Using the Neural Network and the model weights found in the modules "Model" folder, the module first loads a ViperModel object:

In [5]:
source = inspect.getsource(ViperModel)
print (source)

class ViperModel(object):
    # A ViperModel contains the location of the model architecture
    # and the model weights which are stored in the node package.
    
    def __init__(self, package_name, model_xml, weights_bin):
        self.pkg = package_name
        self.model = model_xml
        self.weights = weights_bin
        self.setup_model()
        
    def setup_model(self):
        self.dir = roslib.packages.get_pkg_dir(self.pkg)
        self.location = os.path.join(
            self.dir, 
            self.model
            )
        self.weights = os.path.join(
            self.dir, 
            self.weights
            )



### Instantiating the class NeuralNetworkLoader
When the VPU is initialized, the Model Server will provide the location of these parameters to the inference engine, as well as create a class object for this module called which I call a NeuralNetworkLoader. This object allows the model to be initialized at the time of the VPU instantiation, which is asyncronous to the instantiation of this module:

***(Note: While I am providing the class definition here, the NeuralNetworkLoader Class is shared amongst all Model Nodes; I will break this down how this works in more detail on #TBA page)***

In [8]:
source = inspect.getsource(NeuralNetworkLoader)
print (source)

class NeuralNetworkLoader(object):
    """
    
    A NeuralNetworkLoader is a class object which loads a pretrained
    model (architecture and weights) onto a initialized OpenVino
    inference engine.
    
    Keyword Arguments:
    
    ie -- an Inference Engine instance set up in the parent node which 
    we will load this model to.
    
    ViperModel -- a instance of class ViperModel which contains the 
    models weights and the structure of the neural network.
    
    device -- the inference device to be used to predict on (i.e., 
    "MYRIAD", CPU, GPU, etc.)
    
    model_tag -- a three letter abbreviation used by the VIPER Logger
    module which identifies log messages as originating from within this
    modules code.
    
    model_name -- a logger attribute which identifies this model.
    
    """
    
    def __init__(self, 
                ie: IECore, 
                viper_model: ViperModel,
                device: str,
                model_tag: str = "M..",
    

The effect of this is that the inference engine object is initiated once, and all models are loaded to the inference engine object at initialization. This module then instantiates a SeceneSegmentationModel, which is a child class of the NeuralNetworkLoader already instantiated on behalf of this module. We allow this module to inherit all methods and properties of the parent class.

In effect, the class object that was initiated prior to this modules initialization can then be "passed on" to this module as if it were this module which had instantiated this class.

### Scene Segmentation Model
The SceneSegmentationModel is a class object of type NeuralNetworkLoader, and contains the methods particular to the performance of Scene Segmentation. It contains two class methods:
* export_parameters() which exports the shape of the model specificly needed by this model
* run_scene_segmentation() which is the method by which the image is transformed and passed to the inference engine for prediction. 

After sucessfully invoking this method an image mask is returned with each pixel being classified as one of the four classes.


In [9]:
source = inspect.getsource(SceneSegmentationModel)
print (source)

class SceneSegmentationModel(NeuralNetworkLoader):
    """
    
    A SceneSegmentationModel is a class object which uses a Convolutional
    Neural Network (CNN) to classify  every pixel within an image as a 
    member of a certain class (segments the image). In this  pytorch 
    pretrained model we are predicting on 4 classes: 
        (a) Roadway, (b) Curb, (c) Background, and (d) marker. 
    
    The model specifications can be found at: 
    https://docs.openvino.ai/2018_R5/_docs_Transportation_segmentation_curbs_release1_caffe_desc_road_segmentation_adas_0001.html
    
    """
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def export_parameters(export_state=True, **kwargs):
        # Exports parameters sent to this function for debugging 
        # purposes, and then turns off function after export_state=False
        # is received.
        self.export_state = False
        
        for arg in kwargs:
            self.parameters.ad